# ARI3129 Assignment - 2b - Liam Jake Vella
## Sign Shape Detector - Faster RCNN

In [ ]:
# Install dependencies
!pip install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu126


In [ ]:
!pip install git+https://github.com/facebookresearch/detectron2.git

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-r72ny945
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-r72ny945
  Resolved https://github.com/facebookresearch/detectron2.git to commit fd27788985af0f4ca800bca563acdb700bb890e2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.4/86.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 6.6 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp312-cp312-linux_x86_64.whl size=7084955 sha256=37136d72af8d5cf37ef28294c59f0f102e26b6ad03f87713525da08245940eef
  Stored in d

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

<Logger detectron2 (DEBUG)>

In [ ]:
!wget --user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36" -O /content/COCO_sign_shape.zip "https://dl-bkgz5xt6.swisstransfer.com/api/download/e16331e1-fd02-4a8b-9147-360963603aa0/085709e0-b4c6-4152-8dcf-a71e31743fb6"

--2026-01-17 18:41:22--  https://dl-bkgz5xt6.swisstransfer.com/api/download/e16331e1-fd02-4a8b-9147-360963603aa0/085709e0-b4c6-4152-8dcf-a71e31743fb6
Resolving dl-bkgz5xt6.swisstransfer.com (dl-bkgz5xt6.swisstransfer.com)... 83.166.143.218
Connecting to dl-bkgz5xt6.swisstransfer.com (dl-bkgz5xt6.swisstransfer.com)|83.166.143.218|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1734451941 (1.6G) [binary/octet-stream]
Saving to: ‘/content/COCO_sign_shape.zip’

/content/COCO_sign_ 100%[===================>]   1.62G  18.2MB/s    in 95s     

2026-01-17 18:42:58 (17.5 MB/s) - ‘/content/COCO_sign_shape.zip’ saved [1734451941/1734451941]



In [ ]:
!unzip /content/COCO_sign_shape.zip -d /content

Archive:  /content/COCO_sign_shape.zip
   creating: /content/COCO-based_COCO_sign_shape/
   creating: /content/COCO-based_COCO_sign_shape/annotations/
  inflating: /content/COCO-based_COCO_sign_shape/annotations/test.json  
  inflating: /content/COCO-based_COCO_sign_shape/annotations/train.json  
  inflating: /content/COCO-based_COCO_sign_shape/annotations/val.json  
   creating: /content/COCO-based_COCO_sign_shape/images/
   creating: /content/COCO-based_COCO_sign_shape/images/test/
  inflating: /content/COCO-based_COCO_sign_shape/images/test/0015da15-IMG_0222.JPG  
  inflating: /content/COCO-based_COCO_sign_shape/images/test/04258956-back.jpg  
  inflating: /content/COCO-based_COCO_sign_shape/images/test/0909c314-side.jpg  
  inflating: /content/COCO-based_COCO_sign_shape/images/test/0add08e2-Front.jpg  
  inflating: /content/COCO-based_COCO_sign_shape/images/test/0b0fac4f-front.jpg  
  inflating: /content/COCO-based_COCO_sign_shape/images/test/0c60af5d-side.jpg  
  inflating: /conte

In [ ]:
from detectron2.data.datasets import register_coco_instances

# Register Train
register_coco_instances(
    "sign_train",
    {},
    "/content/COCO-based_COCO_sign_shape/annotations/train.json",
    "/content/COCO-based_COCO_sign_shape/images/train"
)

# Register Val
register_coco_instances(
    "sign_val",
    {},
    "/content/COCO-based_COCO_sign_shape/annotations/val.json",
    "/content/COCO-based_COCO_sign_shape/images/val"
)

In [ ]:
from detectron2.config import get_cfg
from detectron2 import model_zoo

cfg = get_cfg()

# 1. Load a pre-trained model config (ResNet-50 is a good standard backbone)
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

# 2. Set datasets
cfg.DATASETS.TRAIN = ("sign_train",)
cfg.DATASETS.TEST = ("sign_val",) # valid dataset

# 3. Workers (Colab usually gives 2 cores)
cfg.DATALOADER.NUM_WORKERS = 2

# 4. Initialize weights from the Model Zoo (Transfer Learning)
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")

# 5. Hyperparameters
cfg.SOLVER.IMS_PER_BATCH = 6   # Batch size (T4 has 16GB VRAM, 4-8 is usually safe)
cfg.SOLVER.BASE_LR = 0.00025   # Learning rate
cfg.SOLVER.MAX_ITER = 2000
cfg.SOLVER.STEPS = (1500,) # Drop the LR at 75% of the way through
cfg.SOLVER.GAMMA = 0.1     # This makes the LR 0.000025 for the last 500 iters

# 6. ROI Heads (Number of classes)
# IMPORTANT: This must match your number of shape classes (e.g., Circle, Triangle, Square)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5

# 7. Output directory
cfg.OUTPUT_DIR = "/content/output"

In [ ]:
from detectron2.engine import DefaultTrainer

# Create a DefaultTrainer instance with our configuration
trainer = DefaultTrainer(cfg)

# Resume training from the last checkpoint or start fresh if no checkpoint exists
trainer.resume_or_load(resume=False)

# Start training the model
trainer.train()

[01/17 19:35:34 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[01/17 19:35:35 d2.engine.train_loop]: Starting training from iteration 0
[01/17 19:36:47 d2.utils.events]:  eta: 2:00:37  iter: 19  total_loss: 1.633  loss_cls: 1.514  loss_box_reg: 0.1017  loss_rpn_cls: 0.01519  loss_rpn_loc: 0.004244    time: 3.4320  last_time: 3.8124  data_time: 2.0559  last_data_time: 2.1172   lr: 4.9953e-06  max_mem: 7578M
[01/17 19:37:48 d2.utils.events]:  eta: 1:53:58  iter: 39  total_loss: 1.525  loss_cls: 1.39  loss_box_reg: 0.08707  loss_rpn_cls: 0.0191  loss_rpn_loc: 0.007359    time: 3.2478  last_time: 1.2380  data_time: 1.5047  last_data_time: 0.0149   lr: 9.9902e-06  max_mem: 7578M
[01/17 19:38:50 d2.utils.events]:  eta: 1:44:32  iter: 59  total_loss: 1.271  loss_cls: 1.123  loss_box_reg: 0.09626  loss_rpn_cls: 0.01659  loss_rpn_loc: 0.005021    time: 3.1936  last_time: 4.1501  data_time: 1.5415  last_data_time: 2.7387   lr: 1.4985e-05  max_mem: 7579M
[01/17 19:39:52 d2.utils.events]:  eta: 1:40:12  iter: 79  total_loss: 0.9508  loss_cls: 0.8094  loss_bo

In [ ]:
!zip -r fasterrcnnljv.zip /content/output/

  adding: content/output/ (stored 0%)
  adding: content/output/events.out.tfevents.1768678607.06f238252c4c.408.1 (deflated 72%)
  adding: content/output/events.out.tfevents.1768678144.06f238252c4c.408.0 (deflated 65%)
  adding: content/output/metrics.json (deflated 79%)
  adding: content/output/model_final.pth (deflated 7%)
  adding: content/output/last_checkpoint (stored 0%)


In [ ]:
from google.colab import files
files.download('fasterrcnnljv.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>